# Intro to RNNs
This is loosely based on: https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

Imports:

In [45]:
import torch
from torch import nn
import numpy as np
from timeit import default_timer as timer

Helper functions:

In [46]:
def split_eq(text, no):
    cnt = int(len(text) / no)
    examples = [text[i:i+cnt] for i in range(0, len(text), cnt)]
    if (no*cnt == len(text)):
        return examples
    else:
        return examples[:-1]

def produce_targets(examples):
    targets = [ex[1:] for ex in examples]
    inputs = [ex[:-1] for ex in examples]
    return inputs, targets

def translate_to_int(examples):
    translated = [list(map(lambda ch: char2int[ch], ex)) for ex in examples]
    return translated

def translate_to_char(examples):
    translated = [''.join(list(map(lambda i: int2char[i], ex))) for ex in examples]
    return translated

def one_hot_encode(examples):
    features = np.zeros((len(examples), len(examples[0]), len(char2int)), dtype=np.float32)
    
    for i, example in enumerate(examples):
        for pos in range(len(examples[i]) - 1):
            features[i, pos, examples[i][pos]] = 1
    return features

def to_model_format(inputs):
    if isinstance(inputs, str):
        inputs = [inputs]
    trans_inputs = translate_to_int(inputs)
    encoded = one_hot_encode(trans_inputs)
    encoded_tensor = torch.from_numpy(encoded)
    return encoded_tensor

def training_check(output):
    # TODO: fix
    print("#########", len(output))
    single_char = None
    for char_no, out in enumerate(output):
        mx = max(out)
        for no, val in enumerate(out):
            if val == mx and (not single_char or single_char == no):
                print(val, no, int2char[no], int2char[trans_inputs[0][char_no]])

Configurations:

In [62]:
# configuration
no_of_examples = 32
examples_per_batch = 16
n_epochs = 300
lr = 0.0048
no_of_chars = 16000

Loading training data:

In [67]:

with open('text', 'r') as fd:
    full_text = fd.read().lower()
full_text = full_text[0:no_of_chars]

vocab = set(full_text)
int2char = dict(enumerate(vocab))
char2int = {char: ind for ind, char in int2char.items()}
print("Dictionary size:", len(char2int))

Dictionary size: 44


In [68]:
no_of_batches = int(no_of_examples / examples_per_batch)

examples = split_eq(full_text, no_of_examples)
chars_per_example = len(examples[0])
inputs, targets = produce_targets(examples)
trans_inputs = translate_to_int(inputs)
trans_targets = translate_to_int(targets)

batches = []

for i in range(no_of_batches):
    input_seq = one_hot_encode(trans_inputs[i*examples_per_batch:(i+1)*examples_per_batch])
    target_seq = torch.Tensor(trans_targets[i*examples_per_batch:(i+1)*examples_per_batch])
    batches.append((torch.from_numpy(input_seq), target_seq))

print("No of examples/No of data parts:", no_of_examples)
print("No of batches:", no_of_batches)
print("Examples per batch:", examples_per_batch)
print("Chars per example:", chars_per_example)

No of examples/No of data parts: 32
No of batches: 2
Examples per batch: 16
Chars per example: 500


The Model.

It's RNN which uses hidden state and input.

A word on sizes: https://pytorch.org/docs/stable/nn.html#rnn

Hidden state input: (n_layers, batch_size, hidden_size)
These are arbitrarily chosen by us before training.

Data input: (seq_len, batch, input_size) if batch_first=True in RNN init, data input becomes: (batch, seq_len,  input_size)
which I think is more natural.

We have first mini-batches, then examples (a whole sentence in this case: "something something..."), than single one-hot encoded char. We use mini-batches and 'examples_per_batch' defined above is the number of examples in single mini-batch.

Notice also "init_hidden" function. Hidden state is zeroed before every mini-batch i.e. during every forward pass.
This can be done differently. If we have same size of batches we can pass hidden state from previous runs. A pattern here is to init once per epoch.

In [76]:
class ModelRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers):
        super(ModelRNN, self).__init__()
        output_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.rnn = nn.RNN(input_size, hidden_size, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return hidden

Init model:

In [77]:
dict_size = len(char2int)
model = ModelRNN(input_size=dict_size, hidden_size=12, n_layers=3)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Training:

In [78]:
t1 = timer()
for epoch in range(1, n_epochs):
    for no, batch in enumerate(batches):
        inp, target = batch
        optimizer.zero_grad()
        output, hidden = model(inp)
        # training_check(input_seq, output)
        loss = criterion(output, target.view(-1).long())
        loss.backward()
        optimizer.step()

    if epoch%25 == 0:
        print('.',)
    if epoch%100 == 0:
        print('Epoch: {}/{}......'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))
        t2 = timer()
        print("Elapsed:", t2 - t1)
t3 = timer()
print("Total secods elapsed:", t3 - t1)

.
.
.
.
Epoch: 100/300...... Loss: 2.3464
Elapsed: 29.864484400000947
.
.
.
.
Epoch: 200/300...... Loss: 2.0640
Elapsed: 59.872537100000045
.
.
.
Total secods elapsed: 89.47705769999993


Prediction helpers:

In [80]:
def predict_next(model, input_string):
    encoded_input = to_model_format(input_string)
    out, hidden = model(encoded_input)

    # choosing one with highest probability
    prob = nn.functional.softmax(out[-1], dim=0).data
    char_ind = torch.max(prob, dim=0)[1].item()
    return int2char[char_ind]


def run_model(model, starting_seq, size=50):
    model.eval()
    seq = starting_seq.lower()
    for _ in range(size):
        char = predict_next(model, seq)
        seq += char
    return ''.join(seq)

Prediction:

In [81]:
res = run_model(model, 'character ')
print(res)

character euat  rreat  freas  freas  freas  freas  freas  fr


Some results:

ep:1500 lr:0.004 1000chars: "character auddoc aouis thrts cmgei eheo  bwiogib throliaeddo"

ep:1500 lr:0.004 2000chars: "character serlpiip  oa doee  foerdtoitn aad ehd eou  eosu  o" 

ep:1000 lr:0.0045 4000chars: "character iat  iypheae  nimftercteitrtrotd ia .eta aat  aeto"

ep:1000 lr:0.0045 4000chars Inbatch:8 "character aodlt voot  eo csrtoe .p tpucsl\neopdsiuts tautor t"

ep:2400 lr:0.0045 4000chars Inbatch:8 "character ootteos meatcias eopneansen rast aagteos road iet "

ep:1800 lr:0.0045 8000chars Inbatch:32 "character lpian tpratttttttttttttttttttttttttttttttttttttttt", Loss:1.8095

ep:2800 lr:0.0047 8000chars Inbatch:32 "character  au dhr eat  uumsr eottrreent  au dhr eat  uumsr e" Loss:1.8013

ep:2000 lr:0.0048 16000chars Inbatch:16 Batches:2 "character  uhsist rrietsr yho nurtpr  rhaed  uhsist rrietsr " Loss:1.8369